## Solveur SAT

In [2]:
MAX_GRADE = 21
import sys
sys.path.append('../')

from generator import GradesGenerator


In [2]:
size = 100
nb_grades = 5
noise = 0
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()

gen.analyze_gen()


if nb_class == 1:
    admissions = admissions.astype(int)


---------Analyze----------
Lambda: 0.7027020406842801
Weights: [0.21340252 0.00601878 0.12457413 0.35883431 0.29717027]
Betas: [14 13 12  8 10]
Got-in: {False: 78, True: 22}
% Got-in: {False: 78.0, True: 22.0} %
--------------------------


In [32]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [33]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [34]:
#Clause 1
clause_1 = []

for i in range(nb_grades):
    for k in range(0,MAX_GRADE-1):
        for j in range(k+1,MAX_GRADE):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])
         

In [35]:
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])



In [36]:
#Clause 3

clause_3 = []
for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 1:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(v2i_alpha[(i,student[i])])
        
            clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])

clause_3

[[137],
 [16, 136],
 [42, 135],
 [56, 134],
 [81, 133],
 [104, 132],
 [16, 42, 131],
 [16, 56, 130],
 [16, 81, 129],
 [16, 104, 128],
 [42, 56, 127],
 [42, 81, 126],
 [42, 104, 125],
 [56, 81, 124],
 [56, 104, 123],
 [81, 104, 122],
 [16, 42, 56, 121],
 [16, 42, 81, 120],
 [16, 42, 104, 119],
 [16, 56, 81, 118],
 [16, 56, 104, 117],
 [16, 81, 104, 116],
 [42, 56, 81, 115],
 [42, 56, 104, 114],
 [42, 81, 104, 113],
 [56, 81, 104, 112],
 [16, 42, 56, 81, 111],
 [16, 42, 56, 104, 110],
 [16, 42, 81, 104, 109],
 [16, 56, 81, 104, 108],
 [42, 56, 81, 104, 107],
 [16, 42, 56, 81, 104, 106],
 [137],
 [16, 136],
 [37, 135],
 [58, 134],
 [73, 133],
 [100, 132],
 [16, 37, 131],
 [16, 58, 130],
 [16, 73, 129],
 [16, 100, 128],
 [37, 58, 127],
 [37, 73, 126],
 [37, 100, 125],
 [58, 73, 124],
 [58, 100, 123],
 [73, 100, 122],
 [16, 37, 58, 121],
 [16, 37, 73, 120],
 [16, 37, 100, 119],
 [16, 58, 73, 118],
 [16, 58, 100, 117],
 [16, 73, 100, 116],
 [37, 58, 73, 115],
 [37, 58, 100, 114],
 [37, 73, 1

In [37]:
clause_4 = []


for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 0:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i])])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [38]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+A+1)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [39]:
#Lancer la résolution

myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile.cnf")
res = exec_gophersat("./workingfile.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, 16, 17, 18, 19, 20, 21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, 35, 36, 37, 38, 39, 40, 41, 42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, 55, 56, 57, 58, 59, 60, 61, 62, 63, -64, -65, -66, -67, -68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, -106, -107, -108, -109, -110, -111, -112, -113, -114, -115, -116, -117, -118, -119, -120, -121, -122, -123, -124, -125, -126, -127, -128, -129, -130, -131, -132, -133, -134, -135, -136, 137], {(0, 0): False, (0, 1): False, (0, 2): False, (0, 3): False, (0, 4): False, (0, 5): False, (0, 6): False, (0, 7): False, (0, 8): False, (0, 9): False, (0, 10): False, (0, 11): False, (0, 12): False, (0, 13): False, (0, 14): False, (0, 15): True, (0, 16): True, (0, 17): True, (0, 18): True, (0, 19): True, (0, 20): True, (1, 0): False, (1

In [40]:
_, variables_idx, d = res

In [41]:
predicted = []
for student in grades:
    validated_courses = set()
    for i,k in enumerate(student):
        if d[(i,k)]:
            validated_courses.add(i)
    validated_courses = frozenset(validated_courses)
    if d[validated_courses]:
        predicted.append(1)
    else:
        predicted.append(0)
predicted
        


[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [42]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted))

1.0
1.0


SAT Solver Multi classes

In [6]:
size = 100
nb_grades = 5
noise = 0
nb_class = 2

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()


if nb_class == 1:
    admissions = admissions.astype(int)

admissions


array([0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 1, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
grades

array([[19,  5,  9, 11, 11],
       [15,  8, 13, 13,  4],
       [ 0,  0, 18, 10, 10],
       [ 4,  4, 12,  6,  0],
       [12, 16,  2,  7,  4],
       [ 6, 10, 13,  8, 19],
       [11,  4, 17, 11, 20],
       [19,  7,  5, 20,  3],
       [ 0,  7,  4,  4, 19],
       [14,  5,  8,  4, 20],
       [ 0, 11, 11,  6,  0],
       [16, 15, 19, 19, 10],
       [16,  9,  0, 13,  3],
       [10, 19, 12,  4,  3],
       [20, 16, 12, 12, 12],
       [19,  1, 18, 16,  4],
       [13, 19,  6,  3, 16],
       [12, 17, 16, 15,  9],
       [10, 18,  3, 11, 20],
       [ 4, 20,  0,  8, 17],
       [14, 10, 17, 20,  9],
       [ 7,  1, 14,  8, 14],
       [17,  1, 16, 10, 17],
       [17,  6,  5,  8, 13],
       [10, 15,  5,  0,  8],
       [12,  2,  0,  6,  7],
       [18,  3,  2, 14, 10],
       [ 0, 13, 20,  5,  6],
       [ 8, 19,  8,  3,  4],
       [ 4,  7,  1, 12,  3],
       [ 3,  0,  6,  9, 17],
       [ 6,  8, 20, 12, 16],
       [11,  7, 13,  5,  7],
       [ 9,  5, 19, 13,  1],
       [14, 14

In [45]:
admissions

array([1, 0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 2, 2, 0, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 2, 1,
       2, 1, 2, 2, 1, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 0, 0,
       2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0, 2,
       2, 2, 1, 0, 1, 2, 2, 2, 2, 2, 2, 2])

In [46]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        for h in range(nb_class+1):
            alpha.append((i,k,h)) # ==> donne tous les alpha i k h

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [47]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [48]:
clause_1 = []

for i in range(nb_grades):
    for k in range(MAX_GRADE-1):
        for h in range(nb_class):
            for j in range(k+1,MAX_GRADE):
                clause_1.append([-v2i_alpha[(i,k,h)], v2i_alpha[(i,j,h)]])

In [49]:
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])


In [50]:
clause_3 = []


clause_3 = []
for st_idx,student in enumerate(grades):
    for c in subsets:
        alpha = []
        for i in c:
            alpha.append(v2i_alpha[(i,student[i],admissions[st_idx])])
    
        clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])





In [51]:
clause_4 = []

for st_idx,student in enumerate(grades):
    if admissions[st_idx] < nb_class:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i],admissions[st_idx]+1)])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [52]:
clause_5 = []

for k in range(MAX_GRADE):
    for i in range(nb_grades):
        for h in range(nb_class):
            for j in range(h+1, nb_class+1):
                clause_5.append([v2i_alpha[(i,k,h)],-v2i_alpha[(i,k,j)]])



In [53]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+A+1] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    print(lines)

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [54]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4 + clause_5
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile_multi.cnf")
res = exec_gophersat("./workingfile_multi.cnf")

#Résultat

['c solving ./workingfile_multi.cnf', 's SATISFIABLE', 'v -1 -2 -3 -4 -5 -6 7 8 9 10 11 -12 13 14 -15 16 17 -18 19 20 -21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 -45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 -60 61 62 63 64 -65 -66 67 -68 -69 70 -71 -72 73 -74 -75 76 -77 -78 79 -80 -81 82 -83 -84 85 -86 -87 88 -89 -90 91 92 -93 94 95 96 97 98 -99 100 101 -102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 -128 -129 130 -131 -132 133 -134 -135 136 -137 -138 139 -140 -141 142 -143 -144 145 -146 -147 148 -149 -150 151 -152 -153 154 -155 -156 157 -158 -159 160 -161 -162 163 -164 -165 166 -167 -168 169 -170 -171 172 -173 -174 175 -176 -177 178 -179 -180 181 -182 -183 184 -185 -186 187 188 -189 190 191 192 193 194 195 196 197 198 199 200 -201 202 203 204 205 206 207 208 209 -210 211 212 213 214 215 216 217 218 219 220 221 -222 223 224 -225 226 227 228 229 230 231 232 233 234 235 236 -237 238 239 240 241 242 243

In [55]:
_, variables_idx, d = res

In [56]:
predicted = []
for student in grades:
    current_class = 0
    for h in range(nb_class+1):
        current_class = h
        validated_courses = set()
        for i,k in enumerate(student):
            if d[(i,k,h)]:
                validated_courses.add(i)
        validated_courses = frozenset(validated_courses)
        if validated_courses in d.keys():
            if d[validated_courses]:
                continue
            else:
                break
        else:
            break
    predicted.append(current_class-1)

predicted
        


[1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [57]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted, average='macro'))

0.35
0.44329896907216493


MaxSAT Solver for multi class

In [3]:
size = 100
nb_grades = 4
noise = 0.1
nb_class = 3

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()


if nb_class == 1:
    admissions = admissions.astype(int)

admissions = admissions.astype(int)

admissions


Adding 10.00 % of noise


array([0, 0, 0, 3, 2, 1, 0, 3, 3, 3, 1, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 1,
       1, 1, 3, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 3, 0, 3, 2, 3, 0, 1,
       3, 3, 0, 0, 1, 3, 0, 3, 0, 0, 1, 0, 3, 0, 0, 3, 3, 3, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 1, 0, 0, 1, 3, 0, 3, 0, 0, 0, 0, 3, 3, 3,
       1, 0, 0, 1, 2, 3, 3, 3, 3, 3, 3, 3])

In [19]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        for h in range(nb_class+1):
            alpha.append((i,k,h)) # ==> donne tous les alpha i k h

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


clause_weight = {}
clause_index = 0
W = int(1e5)
w = 1

clause_1 = []

for i in range(nb_grades):
    for k in range(MAX_GRADE-1):
        for h in range(nb_class):
            for j in range(k+1,MAX_GRADE):
                clause_1.append([-v2i_alpha[(i,k,h)], v2i_alpha[(i,j,h)]])
                clause_weight[clause_index] = W
                clause_index += 1

clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])
            clause_weight[clause_index] = W
            clause_index += 1

clause_3 = []
for st_idx,student in enumerate(grades):
    for c in subsets:
        alpha = []
        for i in c:
            alpha.append(v2i_alpha[(i,student[i],admissions[st_idx])])
    
        clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])
        clause_weight[clause_index] = w
        clause_index += 1



clause_4 = []

for st_idx,student in enumerate(grades):
    if admissions[st_idx] < nb_class:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i],admissions[st_idx]+1)])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])
            clause_weight[clause_index] = w
            clause_index += 1
            
clause_5 = []

for k in range(MAX_GRADE):
    for i in range(nb_grades):
        for h in range(nb_class):
            for j in range(h+1, nb_class+1):
                clause_5.append([v2i_alpha[(i,k,h)],-v2i_alpha[(i,k,j)]])
                clause_weight[clause_index] = W
                clause_index += 1




In [13]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+A+1] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np wcnf '+str(numvar)+' '+str(len(clauses))+'\n' #wcnf
    for i,clause in enumerate(clauses) :
        dimacs += str(clause_weight[i]) + ' ' #adding weight
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    print(lines)
    
    for i in range(len(lines)):
            if "FOUND" in lines[i]:
                index = i
                break

    model = lines[index+1][2:].split(" ")[:-1]

    x_var = model[0][0]
    print(x_var)

    for i,var in enumerate(model):
        l = list(var)
        if x_var in l:
            l.remove(x_var)
        var = "".join(l)
        model[i] = int(var)

    print(model)

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 

In [14]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4 + clause_5
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile_maxsat.wcnf")
res = exec_gophersat("./workingfile_maxsat.wcnf")

#Résultat

['c solving ./workingfile_maxsat.wcnf', 'o 13200159', 'o 13200158', 'o 9800050', 'o 9800048', 'o 9700125', 'o 9600187', 'o 9600081', 'o 7300029', 'o 7200030', 'o 7100048', 'o 2400104', 'o 2300104', 'o 2300103', 'o 2300093', 'o 102', 'o 100', 'o 43', 'o 42', 'o 41', 'o 40', 'o 39', 'o 37', 'o 35', 'o 34', 'o 33', 'o 32', 'o 27', 'o 26', 'o 24', 'o 23', 'o 22', 'o 21', 'o 20', 'o 19', 'o 18', 's OPTIMUM FOUND', 'v x1 -x2 -x3 -x4 x5 -x6 -x7 -x8 x9 -x10 -x11 -x12 x13 -x14 -x15 -x16 x17 -x18 -x19 -x20 x21 -x22 -x23 -x24 x25 -x26 -x27 -x28 x29 -x30 -x31 -x32 x33 -x34 -x35 -x36 x37 -x38 -x39 -x40 x41 -x42 -x43 -x44 x45 -x46 -x47 -x48 x49 -x50 -x51 -x52 x53 -x54 -x55 -x56 x57 x58 x59 x60 x61 x62 x63 -x64 x65 x66 x67 -x68 x69 x70 x71 x72 x73 x74 x75 x76 x77 x78 x79 x80 x81 x82 x83 x84 x85 -x86 -x87 -x88 x89 -x90 -x91 -x92 x93 -x94 -x95 -x96 x97 -x98 -x99 -x100 x101 -x102 -x103 -x104 x105 -x106 -x107 -x108 x109 -x110 -x111 -x112 x113 -x114 -x115 -x116 x117 -x118 -x119 -x120 x121 -x122 -x123 -x12

In [15]:
_, variables_idx, d = res

In [16]:
d

{(0, 0, 0): True,
 (0, 0, 1): False,
 (0, 0, 2): False,
 (0, 0, 3): False,
 (0, 1, 0): True,
 (0, 1, 1): False,
 (0, 1, 2): False,
 (0, 1, 3): False,
 (0, 2, 0): True,
 (0, 2, 1): False,
 (0, 2, 2): False,
 (0, 2, 3): False,
 (0, 3, 0): True,
 (0, 3, 1): False,
 (0, 3, 2): False,
 (0, 3, 3): False,
 (0, 4, 0): True,
 (0, 4, 1): False,
 (0, 4, 2): False,
 (0, 4, 3): False,
 (0, 5, 0): True,
 (0, 5, 1): False,
 (0, 5, 2): False,
 (0, 5, 3): False,
 (0, 6, 0): True,
 (0, 6, 1): False,
 (0, 6, 2): False,
 (0, 6, 3): False,
 (0, 7, 0): True,
 (0, 7, 1): False,
 (0, 7, 2): False,
 (0, 7, 3): False,
 (0, 8, 0): True,
 (0, 8, 1): False,
 (0, 8, 2): False,
 (0, 8, 3): False,
 (0, 9, 0): True,
 (0, 9, 1): False,
 (0, 9, 2): False,
 (0, 9, 3): False,
 (0, 10, 0): True,
 (0, 10, 1): False,
 (0, 10, 2): False,
 (0, 10, 3): False,
 (0, 11, 0): True,
 (0, 11, 1): False,
 (0, 11, 2): False,
 (0, 11, 3): False,
 (0, 12, 0): True,
 (0, 12, 1): False,
 (0, 12, 2): False,
 (0, 12, 3): False,
 (0, 13, 0): 

In [17]:
predicted = []
for student in grades:
    current_class = 0
    for h in range(nb_class+1):
        current_class = h
        validated_courses = set()
        for i,k in enumerate(student):
            if d[(i,k,h)]:
                validated_courses.add(i)
        validated_courses = frozenset(validated_courses)
        if validated_courses in d.keys():
            if d[validated_courses]:
                continue
            else:
                break
        else:
            break
    predicted.append(current_class-1)

predicted
        


[0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [18]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted, average='macro'))

0.6
0.4478300617518198


MaxSAT for Rejected + Accepted

In [ ]:
size = 100
nb_grades = 5
noise = 0.1
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()

gen.analyze_gen()


if nb_class == 1:
    admissions = admissions.astype(int)


Adding 10.00 % of noise
---------Analyze----------
Lambda: 0.6993513100729705
Weights: [0.12354862 0.43655624 0.04474298 0.21267943 0.18247273]
Betas: [14 13 13 14  9]
Adding 10.00 % of noise
Got-in: {True: 22, False: 78}
% Got-in: {True: 22.0, False: 78.0} %
--------------------------


In [ ]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)

from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}

clause_weight = {}
clause_index = 0
W = int(1e5)
w = 1

#Clause 1
clause_1 = []

for i in range(nb_grades):
    for k in range(0,MAX_GRADE-1):
        for j in range(k+1,MAX_GRADE):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])
            clause_weight[clause_index] = W
            clause_index += 1
         
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])
            clause_weight[clause_index] = W
            clause_index += 1

#Clause 3

clause_3 = []
for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 1:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(v2i_alpha[(i,student[i])])
        
            clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])
            clause_weight[clause_index] = w
            clause_index += 1

clause_3

clause_4 = []


for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 0:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i])])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])
            clause_weight[clause_index] = w
            clause_index += 1


In [ ]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+A+1] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np wcnf '+str(numvar)+' '+str(len(clauses))+'\n' #wcnf
    for i,clause in enumerate(clauses) :
        dimacs += str(clause_weight[i]) + ' ' #adding weight
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    print(lines)

    for i in range(len(lines)):
        if "FOUND" in lines[i]:
            index = i
            break
    
    model = lines[index+1][2:].split(" ")[:-1]
    

    x_var = 'x'


    for i,var in enumerate(model):
        l = list(var)
        if x_var in l:
            l.remove(x_var)
        var = "".join(l)
        model[i] = int(var)

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 

In [65]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile_maxsat.wcnf")
res = exec_gophersat("./workingfile_maxsat.wcnf")

#Résultat

In [ ]:
_, variables_idx, d = res

In [ ]:
d

{(0, 0): False,
 (0, 1): False,
 (0, 2): False,
 (0, 3): False,
 (0, 4): False,
 (0, 5): False,
 (0, 6): False,
 (0, 7): False,
 (0, 8): False,
 (0, 9): False,
 (0, 10): False,
 (0, 11): False,
 (0, 12): False,
 (0, 13): False,
 (0, 14): True,
 (0, 15): True,
 (0, 16): True,
 (0, 17): True,
 (0, 18): True,
 (0, 19): True,
 (0, 20): True,
 (1, 0): False,
 (1, 1): False,
 (1, 2): False,
 (1, 3): False,
 (1, 4): False,
 (1, 5): False,
 (1, 6): False,
 (1, 7): False,
 (1, 8): False,
 (1, 9): False,
 (1, 10): False,
 (1, 11): True,
 (1, 12): True,
 (1, 13): True,
 (1, 14): True,
 (1, 15): True,
 (1, 16): True,
 (1, 17): True,
 (1, 18): True,
 (1, 19): True,
 (1, 20): True,
 (2, 0): True,
 (2, 1): True,
 (2, 2): True,
 (2, 3): True,
 (2, 4): True,
 (2, 5): True,
 (2, 6): True,
 (2, 7): True,
 (2, 8): True,
 (2, 9): True,
 (2, 10): True,
 (2, 11): True,
 (2, 12): True,
 (2, 13): True,
 (2, 14): True,
 (2, 15): True,
 (2, 16): True,
 (2, 17): True,
 (2, 18): True,
 (2, 19): True,
 (2, 20): Tru

In [ ]:
predicted = []
for student in grades:
    validated_courses = set()
    for i,k in enumerate(student):
        if d[(i,k)]:
            validated_courses.add(i)
    validated_courses = frozenset(validated_courses)
    if d[validated_courses]:
        predicted.append(1)
    else:
        predicted.append(0)
predicted
        


[1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1]

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted))

0.9
0.8717948717948718
